In [1]:
# math and system imports
import sys
sys.path.append('../..')
import numpy as np
from PIL import Image
import glob
import shutil
import os
from os.path import join

# PyTorch imports
import torch
from torch import cuda
from torch.autograd import Variable
import torch.nn.functional as F

# own scripts
import acquire_ear_dataset as a
import ds_ear_siamese as des
import transforms_data as td
import helpers
from DLBio.pytorch_helpers import get_device


In [7]:
# central variable hub
class Config():
    NN_SIAMESE = False
    AUTHORIZED = ["falco_len","konrad_von"]
    DEVICE = get_device()

    DATASET_DIR = '../dataset_low_res/'
    VERIFICATION_DIR = '../auth_dataset/unknown-auth'
    MODEL_DIR = './models/model_mobile_ALL_95percent.pt'

    RESIZE_SMALL = True

model = torch.load(Config.MODEL_DIR, Config.DEVICE) 

In [8]:
#Output for NN  
if Config.NN_SIAMESE == True:
    def generate_output( img_in, img_in2 ):
        if cuda.is_available():
            return model(Variable(img_in).cuda(), Variable(img_in2).cuda())
        else:
            return model(Variable(img_in), Variable(img_in2))
else:
# Output for Mobilenet
    def generate_output( img_in ):
        if cuda.is_available():
            return model(Variable(img_in).cuda())
        else:
            return model(Variable(img_in))    

In [4]:
#a.capture_ear_images(amount_pic=10, pic_per_stage=10, is_authentification=True)

In [9]:
# Image processing
# input is the filename and the transforamtion function
def image_pipeline(input_, preprocess):
    input_ = Image.open(input_)
    input_ = input_.convert("L")
    input_ = preprocess(input_)
    input_ = input_.reshape(-1, td.get_resize(Config.RESIZE_SMALL)[0], td.get_resize(Config.RESIZE_SMALL)[1], 1)
    input_ = input_.permute(3, 0, 1, 2)
    
    if cuda.is_available():
        return input_.type('torch.cuda.FloatTensor')
    else:
        return input_.type('torch.FloatTensor')

In [10]:
# Setting up the dataset
def get_triplets(dataset_path, user_name, verif_dataset):
    dataset_classes = helpers.rm_DSStore( os.listdir( dataset_path ) )
    NUM_CLASSES = len(dataset_classes)
    user_imgs = helpers.rm_DSStore( os.listdir(join(dataset_path, user_name)) )
    
    preprocess = td.transforms_siamese_verification( td.get_resize(Config.RESIZE_SMALL) )

    # Triplets list will contain anchor(A), positive(P) and negative(N) triplets.
    triplets = []

    # creating anchor, positive, negative triplets (amount equals taken images)
    for img_dir in helpers.rm_DSStore( os.listdir(verif_dataset) ):
        anchor_dir = join(verif_dataset, img_dir)
        anchor = image_pipeline(anchor_dir, preprocess)

        # choose random image from the alleged user and pass the preprocess strategy
        positive = image_pipeline( join(dataset_path, user_name, user_imgs[np.random.randint(0, len(user_imgs))]), preprocess )
        
        # find a class different from anchor. if same as anchor try again
        neg_class = dataset_classes[np.random.randint(NUM_CLASSES)]
        while neg_class == anchor_dir: 
            neg_class = dataset_classes[np.random.randint(NUM_CLASSES)]
        # list of image file names
        neg_class_imgs = helpers.rm_DSStore( os.listdir(join(dataset_path, neg_class)) )
        # choose random image from the negative and pass the preprocess strategy
        negative = image_pipeline( join(dataset_path, neg_class, neg_class_imgs[np.random.randint(0, len(neg_class_imgs))]), preprocess )

        # append triplet
        triplets.append([anchor, positive, negative])

    return triplets

In [11]:
# pers_to_ver = input('\n Enter name end press <return> ==>  ')
pers_to_ver = helpers.choose_folder(Config.DATASET_DIR)
triplet_list = get_triplets(dataset_path=Config.DATASET_DIR,
                            user_name=pers_to_ver,
                            verif_dataset=Config.VERIFICATION_DIR)

Index   Name                
0       sarah_feh           
1       konrad_von          
2       mila_wol            
3       gregor_spi          
4       johannes_wie        
5       tim_moe             
6       nils_loo            
7       falco_len           
8       jesse_kru           
9       janna_qua           
10      alissa_buh          
11      johannes_boe        


In [12]:
verification_counter = 0
#Scaling factor
a = 0.1
for t in triplet_list:
    if Config.NN_SIAMESE == True:
        match_out1, match_out2 = generate_output(t[0], t[1])
        non_match_out1, non_match_out2 = generate_output(t[0], t[2])
    else:
        match_out1 = generate_output(t[0])
        match_out2 = generate_output(t[1])                 
        non_match_out1 = generate_output(t[0])
        non_match_out2 = generate_output(t[2])

    euclidean_distance_pp = F.pairwise_distance(match_out1, match_out2)
    euclidean_distance_pn = F.pairwise_distance(non_match_out1, non_match_out2)

    if(euclidean_distance_pp + a < euclidean_distance_pn): verification_counter += 1

    # format variables
    fmt_id = '{:<12}'
    fmt_eucl = '{:<.3f}'
    print(fmt_id.format('pos-pos: '), fmt_eucl.format( euclidean_distance_pp.item()) )
    print(fmt_id.format('pos-neg: '),fmt_eucl.format( euclidean_distance_pn.item()) )
    print(fmt_id.format('Acc. count: '), '{:>.0f}'.format(verification_counter), '\n')

TypeError: forward() missing 1 required positional argument: 'input2'

In [8]:
NUMBER_AUTHORIZED = int(.8*len(triplet_list))
if  pers_to_ver in Config.AUTHORIZED and verification_counter >= NUMBER_AUTHORIZED:
    print("Access granted! Welcome "  + pers_to_ver + "!")
else:
    print("Access denied")

Access denied


In [10]:
shutil.rmtree('../auth_dataset/unknown-auth')